<a href="https://colab.research.google.com/github/baizhankyzy/female-directors/blob/main/Project_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdflib


RDF GRaph

In [1]:
from rdflib import ConjunctiveGraph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
from urllib.parse import quote
import requests
from google.colab import files

# Fetch the JSON data
url = "https://raw.githubusercontent.com/baizhankyzy/female-directors/refs/heads/main/dataset.json"
response = requests.get(url)
data = response.json()['results']['bindings']

# Initialize the graph
g = ConjunctiveGraph()

# Define namespaces
wd = Namespace("http://www.wikidata.org/entity/")
prop = Namespace("http://www.wikidata.org/prop/direct/")
ex = Namespace("http://example.org/")

# Add data to the graph without blank nodes
for item in data:
    film = URIRef(item['film']['value'])
    director = URIRef(item['director']['value'])
    release_date = Literal(item['releaseDate']['value'], datatype=RDFS.Literal)
    film_label = Literal(item['filmLabel']['value'], lang='en')
    director_label = Literal(item['directorLabel']['value'], lang='en')

    genre_label = item.get('genreLabel', {}).get('value', "Unknown Genre")
    genre_uri = URIRef(f"http://example.org/genre/{quote(genre_label)}")

    country_label = item.get('countryLabel', {}).get('value', "Unknown Country")
    country_uri = URIRef(f"http://example.org/country/{quote(country_label)}")

    # Add RDF triples
    g.add((film, RDF.type, wd.Q11424))  # Film type
    g.add((film, RDFS.label, film_label))
    g.add((film, prop.P57, director))
    g.add((film, prop.P577, release_date))
    g.add((film, prop.P136, genre_uri))
    g.add((film, prop.P19, country_uri))
    g.add((director, RDFS.label, director_label))
    g.add((genre_uri, RDFS.label, Literal(genre_label, lang='en')))
    g.add((country_uri, RDFS.label, Literal(country_label, lang='en')))

# Serialize the graph
output_file = "fixed_output.nq"
g.serialize(output_file, format="nquads")

# Download the fixed file
files.download(output_file)
print(f"Data saved as {output_file} and ready for download.")

<ipython-input-1-257d97776e00>:13: DeprecationWarning: ConjunctiveGraph is deprecated, use Dataset instead.
  g = ConjunctiveGraph()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data saved as fixed_output.nq and ready for download.


# Loading the graph

In [2]:
from rdflib import Dataset

# Load the N-Quads file
graph = Dataset()
graph.parse("fixed_output.nq", format="nquads")
print(f"Graph loaded with {len(graph)} triples.")

Graph loaded with 119821 triples.


## Research questions
How has the number of women directors in feature films changed over the last 30 years, including fluctuations during specific periods and potential external factors such as societal events or industry shifts?


In [3]:
query = """
SELECT ?film ?filmLabel ?director ?directorLabel
WHERE {
    ?film rdf:type <http://www.wikidata.org/entity/Q11424> ;
          rdfs:label ?filmLabel ;
          <http://www.wikidata.org/prop/direct/P57> ?director .
    ?director rdfs:label ?directorLabel .
}
LIMIT 10
"""
results = g.query(query)

for row in results:
    print(f"Film: {row['filmLabel']}, Director: {row['directorLabel']}")

Film: Erotique, Director: Monika Treut
Film: Erotique, Director: Lizzie Borden
Film: Erotique, Director: Clara Law
Film: Kommt Mausi raus?!, Director: Angelina Maccarone
Film: Amigomío, Director: Jeanine Meerapfel
Film: Lou n’a pas dit non, Director: Anne-Marie Miéville
Film: Mixed Nuts, Director: Nora Ephron
Film: Reunion, Director: Lee Grant
Film: Seasons of the Heart, Director: Lee Grant
Film: De Generazione, Director: Asia Argento


In [6]:
query = """
SELECT ?filmLabel ?releaseDate
WHERE {
    ?film <http://www.wikidata.org/prop/direct/P577> ?releaseDate .

    ?film rdfs:label ?filmLabel .
    FILTER(LANG(?filmLabel) = "en")
    FILTER(STR(?releaseDate) < "2000-01-01T00:00:00Z")
}
LIMIT 10
"""
results = g.query(query)

for row in results:
    print(f"Film: {row['filmLabel']}, Release Date: {row['releaseDate']}")

Film: Erotique, Release Date: 1994-01-01T00:00:00Z
Film: Erotique, Release Date: 1994-09-12T00:00:00Z
Film: Erotique, Release Date: 1994-09-30T00:00:00Z
Film: Erotique, Release Date: 1994-10-27T00:00:00Z
Film: Erotique, Release Date: 1995-01-20T00:00:00Z
Film: Kommt Mausi raus?!, Release Date: 1994-01-01T00:00:00Z
Film: Amigomío, Release Date: 1994-01-01T00:00:00Z
Film: Amigomío, Release Date: 1995-06-08T00:00:00Z
Film: Lou n’a pas dit non, Release Date: 1994-01-01T00:00:00Z
Film: Mixed Nuts, Release Date: 1994-01-01T00:00:00Z
